In [2]:
%cd ..

/home/romet/projects/ut/wp4/nvidia-e2e


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import random
import cv2


import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

from tqdm.auto import tqdm

from dataloading.nvidia import NvidiaCropWide, Normalize, NvidiaDataset
from trainer import Trainer

import tensorrt

#%load_ext autoreload
#%autoreload 2

In [4]:
xmin = 300
xmax = 1620

ymin = 520
ymax = 864

height = ymax - ymin
width = xmax - xmin

scale = 0.2
scaled_width = int(scale*width)
scaled_height = int(scale*height)

def crop(img):
    return img[ymin:ymax, xmin:xmax, :]

def resize(img):
    return cv2.resize(img, dsize=(scaled_width, scaled_height), interpolation=cv2.INTER_LINEAR)

def normalise(img):
    return (img / 255)

def preprocess(img):
    img = crop(img)
    img = resize(img)
    img = normalise(img)
    return img

def read_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = preprocess(img)
    return img

def create_tensor(img):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
    img_tensor = torch.FloatTensor(img).to(device).permute(2, 0, 1).unsqueeze(0)
    return img_tensor

In [5]:
# image_path = validset.image_paths[0]
# img = read_image(image_path)
# plt.imshow(img)

In [6]:
steering_model_path = "models/1-pilotnet-steering-v1"
tensorrt_steering_model = TensorrtModel(f"{steering_model_path}/best.onnx")

speed_model_path = "models/1-pilotnet-speed-v1"
tensorrt_speed_model = TensorrtModel(f"{speed_model_path}/best.onnx")

NameError: name 'TensorrtModel' is not defined

In [ ]:
root_path = Path("/home/romet/data2/datasets/rally-estonia/dataset")
valid_paths = [root_path / "2021-06-07-14-20-07_e2e_rec_ss6"]
validset = NvidiaDataset(valid_paths, camera="front_wide")

In [ ]:
steering_predictions = []
speed_predictions = []

for path in tqdm(validset.frames.image_path.to_numpy()):
    img = read_image(path)
    img = np.transpose(img, [2, 0, 1])
    img = np.expand_dims(img, axis=0)
    img = np.array(img, dtype=np.float32, order='C')
    steering_predictions.append(tensorrt_steering_model.predict(img))
    speed_predictions.append(tensorrt_speed_model.predict(img))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.plot(validset.steering_angles)
ax1.plot(steering_predictions)
ax1.legend(["ground truth", "predictions"])
ax1.set_title("Steering angle")

ax2.plot(validset.vehicle_speed)
ax2.plot(speed_predictions)
ax2.legend(["ground truth", "predictions"])
ax2.set_title("Vehicle speed");

In [ ]:
import math
import cv2
import shutil
from skimage import io
import os
from moviepy.editor import ImageSequenceClip


def draw_steering_angle(frame, steering_angle, steering_wheel_radius, steering_position, size, color):
    steering_angle_rad = math.radians(steering_angle)
    x = steering_wheel_radius * np.cos(np.pi / 2 + steering_angle_rad)
    y = steering_wheel_radius * np.sin(np.pi / 2 + steering_angle_rad)
    cv2.circle(frame, (steering_position[0] + int(x), steering_position[1] - int(y)), size, color, thickness=-1)
    
def draw_frames(dataset, predicted_angles, predicted_speed, temp_frames_folder):
    
    for frame_index, data in tqdm(enumerate(dataset), total=len(dataset)):
        
        frame = data["image"].permute(1, 2, 0).cpu().numpy()
        true_angle = math.degrees(data["steering_angle"])
        pred_angle = math.degrees(predicted_angles[frame_index])
        
        true_speed = data["vehicle_speed"] * 3.6
        pred_speed = predicted_speed[frame_index] * 3.6
        
        cv2.putText(frame, 'True: {:.2f} deg, {:.2f} km/h'.format(true_angle, true_speed), (10, 1150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)
        cv2.putText(frame, 'Pred: {:.2f} deg, {:.2f} km/h'.format(pred_angle, pred_speed), (10, 1200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2,
                    cv2.LINE_AA)
    
        radius = 200
        steering_pos = (960, 1200)
        cv2.circle(frame, steering_pos, radius, (255, 255, 255), 7)
        
        cv2.rectangle(frame, (905, 1200), (955, 1200-int(3*true_speed)), (0,255,0), cv2.FILLED)
        cv2.rectangle(frame, (965, 1200), (1015, 1200-int(3*pred_speed)), (255, 0,0), cv2.FILLED)
        

        draw_steering_angle(frame, true_angle, radius, steering_pos, 13, (0, 255, 0))
        draw_steering_angle(frame, pred_angle, radius, steering_pos, 9, (255, 0, 0))
            
        io.imsave(f"{temp_frames_folder}/{frame_index + 1:05}.jpg", frame)
        

def convert_frames_to_video(frames_folder, output_video_path, fps=25):
    output_folder = Path(os.path.split(output_video_path)[:-1][0])
    output_folder.mkdir(parents=True, exist_ok=True)

    p = Path(frames_folder).glob('**/*.jpg')
    image_list = sorted([str(x) for x in p if x.is_file()])

    print("Creating video {}, FPS={}".format(frames_folder, fps))
    clip = ImageSequenceClip(image_list, fps=fps)
    clip.write_videofile(output_video_path)

In [ ]:
temp_frames_folder = Path("./temp_frames")
shutil.rmtree(temp_frames_folder, ignore_errors=True)
temp_frames_folder.mkdir()

draw_frames(validset, steering_predictions, speed_predictions, temp_frames_folder)
output_video = "output/test.mp4"
convert_frames_to_video(temp_frames_folder, output_video, fps=30.0)

shutil.rmtree(temp_frames_folder, ignore_errors=True)

In [ ]:
from IPython.display import HTML


HTML(f"""
<video width="640" height="480" controls>
  <source src="{output_video}" type="video/mp4">
</video>
""")

In [ ]:
output_video